# core

> Fill in a module description here

In [ ]:
#| default_exp core

In [ ]:
#| export
from collections import defaultdict
from fastcore.all import *
from faststripe.endpoints import eps
from inspect import Parameter, Signature

import httpx, re

In [ ]:
#| hide
from nbdev import *

import os

In [ ]:
#| exports
stripe_api_url = 'https://api.stripe.com'

In [ ]:
eps[0]


{
    'data': [],
    'op_id': 'GetAccount',
    'path': '/v1/account',
    'qparams': [
        {
            'description': 'Specifies which fields in the response should be expanded.',
            'name': 'expand'
        }
    ],
    'summary': 'Retrieve account',
    'verb': 'get'
}

In [ ]:
ep = first(eps, lambda x: x['data'] and len(x['data']) < 3)
ep


{
    'data': [
        {
            'annotation': <class 'list'>,
            'default': None,
            'description': 'Specifies which fields in the response should be expanded.',
            'name': 'expand'
        },
        {
            'annotation': <class 'bool'>,
            'default': None,
            'description': "To request a new capability for an account, pass true. There can be a delay before the requested capability becomes active. If the capability has any activation requirements, the response includes them in the `requirements` arrays.\n\nIf a capability isn't permanent, you can remove it from the account by passing false. Some capabilities are permanent after they've been requested. Attempting to remove a permanent capability returns an error.",
            'name': 'requested'
        }
    ],
    'op_id': 'PostAccountsAccountCapabilitiesCapability',
    'path': '/v1/accounts/{account}/capabilities/{capability}',
    'qparams': [],
    'summary': 'Update an A

Now, with each of these descriptions, we can easily create a request that we want on the fly. However, to make it a little bit nicer to use in a library, we'll go ahead and automatically generate classes with proper signatures and docstrings that are then easily accessible in any standard IDE.

In [ ]:
#|export
def _mk_param(name, **kwargs):
    kwargs.pop('description', None)
    return Parameter(name, kind=Parameter.POSITIONAL_OR_KEYWORD, **kwargs)

def _mk_sig(req_args, opt_args, anno_args):
    'req_args are args inside paths such as {account}, opt_args are query params, anno_args are payload params'
    params =  [_mk_param(k) for k in req_args]
    params += [_mk_param(**p) for p in opt_args + anno_args]
    return Signature(params)

In [ ]:
path, *_ = partial_format(ep['path'])
req_args = stringfmt_names(path)
opt_args = ep['qparams']
anno_args = ep['data']

_mk_sig(req_args, opt_args, anno_args)

<Signature (account, capability, expand: list = None, requested: bool = None)>

For organizing our APIs into resource groups, which the OpenAPI spec tends to do, we're going to parse the operation ID.

In [ ]:
#|export
def op2nm(op_id):
    'Parse the operation ID to get the resource and name'
    parts = re.findall(r'[A-Z][a-z]*', op_id)
    verb, *nm = [p.lower() for p in parts]
    res, nm = nm[0], nm[1:]
    nm += [verb]
    return res, nm

In [ ]:
op2nm(ep['op_id'])

('accounts', ['account', 'capabilities', 'capability', 'post'])

In [ ]:
#|export
class _OAPIObj: pass

In [ ]:
#| export
def _flatten_data(data, prefix=''):
    'Flatten a dictionary of data so that it can be used in a request body.'
    result = {}
    for k,v in data.items():
        key = f'{prefix}[{k}]' if prefix else k
        if isinstance(v, dict): result.update(_flatten_data(v, key))
        elif isinstance(v, list): 
            for i,item in enumerate(v):
                if isinstance(item, dict): result.update(_flatten_data(item, f'{key}[{i}]'))
                else: result[f'{key}[{i}]'] = item
        else: result[key] = v
    return result

In [ ]:
stripe_key = os.environ['STRIPE_SECRET_KEY']
headers = {'Authorization': f'Bearer {stripe_key}'}

In [ ]:
#|export
names = lambda x: [o['name'] for o in x]

class _OAPIVerb(_OAPIObj):
    __slots__ = 'path verb res name summary pparams qparams url data hdrs client __doc__'.split()
    def __init__(self, path, verb, op_id, summary, qparams, data, url, hdrs, client, kwargs={}):
        res, name = op2nm(op_id) # custom per oapi spec
        name = '_'.join(name)
        path, *_ = partial_format(path, **kwargs)
        pparams = stringfmt_names(path) # params inside path
        param_docs = '\n'.join(f"    {p['name']}: {p['description']}" for p in qparams + data)
        __doc__ = f"{summary}\n\nParameters:\n{param_docs}" if param_docs else summary
        store_attr()

    def __call__(self, *args, **kwargs):
        'Call the API endpoint with the given arguments'
        flds = self.pparams + names(self.qparams) + names(self.data)
        for a, b in zip(args, flds): kwargs[b] = a
        route_p, query_p, data_p = [{p: kwargs[p] for p in o if p in kwargs}
                                    for o in (self.pparams, names(self.qparams), names(self.data))]
        return self.client(self.path, self.verb, route=route_p, query=query_p, data=data_p)

    def __str__(self): return f'{self.res}.{self.name}{signature(self)}'
    @property
    def __signature__(self): return _mk_sig(self.pparams, self.qparams, self.data)
    __call__.__signature__ = __signature__

    def _repr_markdown_(self):
        return f'{self.res}.{self.name}{self.__signature__}: *{self.summary}*'
    __repr__ = _repr_markdown_

With this, we can now construct a entire OpenAPI verb with proper documentation and nice repr inside Jupyter environments!

In [ ]:
v = _OAPIVerb(url=stripe_api_url, hdrs=headers, client=None, **eps[0])
v

account.get(expand): *Retrieve account*

In [ ]:
v?

Signature:      v(expand)
Type:           _OAPIVerb
Docstring:     
Retrieve account

Parameters:
    expand: Specifies which fields in the response should be expanded.
Call docstring: Call the API endpoint with the given arguments

And we can directly call it to perform whatever action this endpoint give us:

In [ ]:
v().keys()

dict_keys(['id', 'object', 'business_type', 'capabilities', 'charges_enabled', 'country', 'default_currency', 'details_submitted', 'payouts_enabled', 'settings', 'type'])

Let's go ahead and group our verbs based on the resource they are apart of. We can setup a repr to make discoverability of operations you can perform easy.

In [ ]:
#|export
class _OAPIVerbGroup(_OAPIObj):
    def __init__(self, name, verbs):
        self.name,self.verbs = name,verbs
        for o in verbs: setattr(self, o.name, o)
    def __str__(self): return "\n".join(str(v) for v in self.verbs)
    def _repr_markdown_(self): return "\n".join(f'- {v._repr_markdown_()}' for v in self.verbs)

In [ ]:
verbs = L(eps).map(lambda x: _OAPIVerb(**x, url=stripe_api_url, hdrs=headers, client=None))
groups = {k.replace('-','_'): _OAPIVerbGroup(k,v) for k,v in groupby(verbs, 'res').items()}
res, g = first(groups.items())
g

- account.get(expand): *Retrieve account*
- account.links_post(account: str = None, collect: str = None, collection_options: dict = None, expand: list = None, refresh_url: str = None, return_url: str = None, type: str = None): *Create an account link*
- account.sessions_post(account: str = None, components: dict = None, expand: list = None): *Create an Account Session*

Let's go ahead and design a class since we need to store our API key and for use in the headers for each of these classes.

In [ ]:
#| export
class StripeApi:
    def __init__(self, api_key=None, base_url=stripe_api_url):
        self.api_key,self.base_url = api_key,base_url
        self.hdrs = {'Authorization': f'Bearer {self.api_key}'}
        verbs = L(eps).map(lambda x: _OAPIVerb(**x, url=base_url, hdrs=self.hdrs, client=self))
        self.func_dict = {f'{o.path}:{o.verb.upper()}':o for o in verbs}
        self.groups = {k.replace('-','_'): _OAPIVerbGroup(k,v) for k,v in groupby(verbs, 'res').items()}
    
    def __call__(self, path:str, verb:str=None, headers:dict=None, route:dict=None, query:dict=None, data=None):
        'Call the API endpoint with the given arguments'
        headers = {**self.hdrs, **(headers or {})}
        if route:
            for k,v in route.items(): route[k] = quote(str(route[k]))
        if data: data = _flatten_data(data)
        res, self.recv_hdrs = urlsend(self.base_url + '/' + path, verb, headers=headers, return_headers=True,
                                      route=route or None, query=query or None, data=data or None, return_json=True,
                                      json_data=False)
        return dict2obj(res)
    
    def __dir__(self): return super().__dir__() + list(self.groups)
    def _repr_markdown_(self): return "\n".join(f"- {o}" for o in sorted(self.groups))
    def __getattr__(self,k): return self.groups[k] if 'groups' in vars(self) and k in self.groups else stop(AttributeError(k))
    def __getitem__(self, k):
        "Lookup and call an endpoint by path and verb (which defaults to 'GET')"
        a,b = k if isinstance(k,tuple) else (k,'GET')
        return self.func_dict[f'{a}:{b.upper()}']

In [ ]:
sapi = StripeApi(os.environ['STRIPE_SECRET_KEY'])
sapi

- account
- accounts
- apple
- application
- apps
- balance
- billing
- charges
- checkout
- climate
- confirmation
- country
- coupons
- credit
- customer
- customers
- disputes
- entitlements
- ephemeral
- events
- exchange
- external
- file
- files
- financial
- forwarding
- identity
- invoice
- invoiceitems
- invoices
- issuing
- link
- linked
- mandates
- payment
- payouts
- plans
- prices
- products
- promotion
- quotes
- radar
- refunds
- reporting
- reviews
- setup
- shipping
- sigma
- sources
- subscription
- subscriptions
- tax
- terminal
- test
- tokens
- topups
- transfers
- treasury
- webhook

In [ ]:
sapi.account

- account.get(expand): *Retrieve account*
- account.links_post(account: str = None, collect: str = None, collection_options: dict = None, expand: list = None, refresh_url: str = None, return_url: str = None, type: str = None): *Create an account link*
- account.sessions_post(account: str = None, components: dict = None, expand: list = None): *Create an Account Session*

In [ ]:
sapi.account.get().keys()

dict_keys(['id', 'object', 'business_type', 'capabilities', 'charges_enabled', 'country', 'default_currency', 'details_submitted', 'payouts_enabled', 'settings', 'type'])

You can also call `StripeApi` directly with the path, verb, parameters, headers, etc:

In [ ]:
prod = sapi('/v1/products', 'POST', data=dict(name='Test Product'))
prod.id, prod.name

('prod_SWtCGrjnxwa0wp', 'Test Product')

In [ ]:
show_doc(StripeApi.__getitem__)

---

### StripeApi.__getitem__

>      StripeApi.__getitem__ (k)

*Lookup and call an endpoint by path and verb (which defaults to 'GET')*

You can access endpoints by indexing into the object. When using the API this way, you do not need to specify what type of parameter (route, query, or post data) is being used. This is, therefore, the same call as above:

In [ ]:
prod = sapi['/v1/products'](name='Test Product') # defaults to GET
prod.data[0].id, prod.data[0].name

('prod_SWtCGrjnxwa0wp', 'Test Product')

That is all we need in order to have a fully functional Python SDK that is compliant with the Stripe OpenAPI spec. Kind of insane that in under 100 lines of code, we can get this functionality, which in my opinion is in some respects even better than the official Stripe Python SDK for the simple fact that we can see the parameters that the functions take without looking up the API reference doc online.

Let's go ahead and try to build with this thing. The simplest payment system that you can have in Stripe is a one-time payment URL. Here is how we do this in our new API. First, we have to create a product and its price.

In [ ]:
prod = sapi.products.post(name='Test Product')
prod.id, prod.name

('prod_SWtKvW6RBrtNWL', 'Test Product')

In [ ]:
price = sapi.prices.post(product=prod.id, unit_amount=10_00, currency='usd')
price.id, price.unit_amount, price.currency

('price_1RbpXeKGhqIw9PXmtfdsDjUh', 1000, 'usd')

Now we can create our checkout session with a mode of payment which means that it will only happen once and is not part of any sort of subscription.

In [ ]:
sapi.checkout

- checkout.sessions_get(created, customer, customer_details, ending_before, expand, limit, payment_intent, payment_link, starting_after, status, subscription): *List all Checkout Sessions*
- checkout.sessions_post(adaptive_pricing: dict = None, after_expiration: dict = None, allow_promotion_codes: bool = None, automatic_tax: dict = None, billing_address_collection: str = None, cancel_url: str = None, client_reference_id: str = None, consent_collection: dict = None, currency: str = None, custom_fields: list = None, custom_text: dict = None, customer: str = None, customer_creation: str = None, customer_email: str = None, customer_update: dict = None, discounts: list = None, expand: list = None, expires_at: int = None, invoice_creation: dict = None, line_items: list = None, locale: str = None, metadata: dict = None, mode: str = None, optional_items: list = None, payment_intent_data: dict = None, payment_method_collection: str = None, payment_method_configuration: str = None, payment_method_data: dict = None, payment_method_options: dict = None, payment_method_types: list = None, permissions: dict = None, phone_number_collection: dict = None, redirect_on_completion: str = None, return_url: str = None, saved_payment_method_options: dict = None, setup_intent_data: dict = None, shipping_address_collection: dict = None, shipping_options: list = None, submit_type: str = None, subscription_data: dict = None, success_url: str = None, tax_id_collection: dict = None, ui_mode: str = None, wallet_options: dict = None): *Create a Checkout Session*
- checkout.sessions_session_get(session, expand): *Retrieve a Checkout Session*
- checkout.sessions_session_post(session, collected_information: dict = None, expand: list = None, metadata: object = None, shipping_options: object = None): *Update a Checkout Session*
- checkout.sessions_session_expire_post(session, expand: list = None): *Expire a Checkout Session*
- checkout.sessions_session_line_items_get(session, ending_before, expand, limit, starting_after): *Retrieve a Checkout Session's line items*

In [ ]:
checkout = sapi.checkout.sessions_post(mode='payment', line_items=[dict(price=price.id, quantity=1)],
                                       success_url='https://localhost:5001/success', cancel_url='https://localhost:5001/cancel')
print(f'Payment link: {checkout.url[:64]}...')

Payment link: https://billing.answer.ai/c/pay/cs_test_a1Ol7CKLturIxauF9uh2vbKD...


Let's make this process even easier by add a higher level api ontop of our StripeApi.

First, let's make it a little bit easier to find an existing products and prices.

In [ ]:
#| export
@patch
def find_product(self:StripeApi, name: str):
    'Find a product by name'
    prods = L(self.products.get().data)
    return first(prods, lambda p: p.name == name)

In [ ]:
sapi.find_product('Test Product')

```json
{ 'active': True,
  'attributes': [],
  'created': 1750366704,
  'default_price': None,
  'description': None,
  'id': 'prod_SWtKvW6RBrtNWL',
  'images': [],
  'livemode': False,
  'marketing_features': [],
  'metadata': {},
  'name': 'Test Product',
  'object': 'product',
  'package_dimensions': None,
  'shippable': None,
  'statement_descriptor': None,
  'tax_code': None,
  'type': 'service',
  'unit_label': None,
  'updated': 1750366704,
  'url': None}
```

In [ ]:
#| export
@patch
def find_prices(self:StripeApi, product_id: str):
    'Find all prices associated with a product id'
    return L(self.prices.get().data).filter(lambda p: p.product == product_id)

In [ ]:
sapi.find_prices(sapi.find_product('Test Product').id)

[{'id': 'price_1RbpXeKGhqIw9PXmtfdsDjUh', 'object': 'price', 'active': True, 'billing_scheme': 'per_unit', 'created': 1750366718, 'currency': 'usd', 'custom_unit_amount': None, 'livemode': False, 'lookup_key': None, 'metadata': {}, 'nickname': None, 'product': 'prod_SWtKvW6RBrtNWL', 'recurring': None, 'tax_behavior': 'unspecified', 'tiers_mode': None, 'transform_quantity': None, 'type': 'one_time', 'unit_amount': 1000, 'unit_amount_decimal': '1000'}]

In [ ]:
#| export
@patch
def priced_product(self:StripeApi, product_name, amount_cents, currency='usd', recurring=None, description=None):
    "Create a product and price if they don't exist"
    prod_params = dict(name=product_name)
    if description: prod_params['description'] = description
    prod = self.find_product(product_name) or self.products.post(**prod_params)
    price_params = dict(product=prod.id, unit_amount=amount_cents, currency=currency)
    if recurring: price_params['recurring'] = recurring
    price = first(self.find_prices(prod.id)) or self.prices.post(**price_params)
    return prod, price

In [ ]:
prod, price = sapi.priced_product('Test Product', 10_00, 'usd')
prod.name, price.id, price.unit_amount

('Test Product', 'price_1RbpXeKGhqIw9PXmtfdsDjUh', 1000)

Now we can automatically create a product or use an existing one when create a one time payment link.

In [ ]:
#| export
@patch
def one_time_payment(self:StripeApi, product_name, amount_cents, success_url, cancel_url, currency='usd', quantity=1, **kw):
    'Create a simple one-time payment checkout'
    _, price = self.priced_product(product_name, amount_cents, currency)
    return self.checkout.sessions_post(mode='payment', line_items=[dict(price=price.id, quantity=quantity)],
                                       automatic_tax={'enabled': True}, success_url=success_url, cancel_url=cancel_url, **kw)

In [ ]:
checkout = sapi.one_time_payment('Test Product', 10_00, 'https://localhost:5001/success', 'https://localhost:5001/cancel', 'usd')
print(f'Payment link: {checkout.url[:64]}...')

Payment link: https://billing.answer.ai/c/pay/cs_test_a106yCKxaLyMefsDHkPoPknP...


Another common use case is subscriptions. Let's go ahead and create a version that makes subscription creation just as easy.

In [ ]:
#| export
@patch
def subscription(self:StripeApi, product_name, amount_cents, success_url, cancel_url,
                 currency='usd', interval='month', **kw):
    'Create a simple recurring subscription'
    _, price = self.priced_product(product_name, amount_cents, currency, recurring=dict(interval=interval))
    return self.checkout.sessions_post(mode='subscription', success_url=success_url, cancel_url=cancel_url,
                                       line_items=[dict(price=price.id, quantity=1)], **kw)

In [ ]:
sub_checkout = sapi.subscription('Test Subscription Product', 10_00, 'https://localhost:5001/success', 'https://localhost:5001/cancel')
print(f'Payment link: {sub_checkout.url[:64]}...')

Payment link: https://billing.answer.ai/c/pay/cs_test_a1qojLG3EpnNajEwCPgm44lR...


**Note:** You'll want to use Stripe's webhook functionality for detecting payment and subscription events. To do so, you'll utilize the Python Stripe SDK that the event actually came from stripe.

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()